UCI 의 human Activity Recognition 데이터셋의 columns을 응용하여 새로운 파생 변수를 생성하고자 함.  

- mean(): Mean value
- std(): Standard deviation
- mad(): Median absolute deviation 
- max(): Largest value in array
- min(): Smallest value in array
- energy(): Energy measure. Sum of the squares divided by the number of values. 
- iqr(): Interquartile range 
- entropy(): Signal entropy
- maxInds(): index of the frequency component with largest magnitude
- meanFreq(): Weighted average of the frequency components to obtain a mean frequency
- skewness(): skewness of the frequency domain signal 
- kurtosis(): kurtosis of the frequency domain signal 
- angle(): Angle between to vectors.


x, y, z 의 결합된 계산 
- sma(): Signal magnitude area
- correlation(): correlation coefficient between two signals 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

train = pd.read_csv('train_features.csv')
train_label = pd.read_csv('train_labels.csv')
test= pd.read_csv('test_features.csv')
submission = pd.read_csv('sample_submission.csv')

In [20]:
pd.set_option('display.max_columns', None)

In [2]:
gb_train = train.drop('time', axis = 1, inplace =False).groupby('id')
gb_test = test.drop('time',axis =1, inplace =False).groupby('id')

In [14]:
def mad(x):
    return np.median(np.absolute(x - np.median(x, axis=0)), axis=0)

In [22]:
def energy(x): # enengy
    x = np.array(x)
    return np.sum(np.square(x)) / len(x)

In [6]:
def iqr(x):
    return np.subtract(*np.percentile(x, [75, 25]))

In [18]:
def entropy(x):
    p = x/x.sum()
    
    return (-p*np.log2(p)).sum()

In [9]:
def skewness(x):
    from scipy.stats import skew 
    
    return skew(x)

def kurtosis(x):
    from scipy.stats import kurtosis 
    return kurtosis(x)

In [11]:
def angleBet(u, v):
    from numpy import (array, dot, arccos, clip)
    from numpy.linalg import norm

    c = dot(u,v)/norm(u)/norm(v) # -> cosine of the angle
    angle = arccos(clip(c, -1, 1)) # if you really want the angle
    
    return angle

In [43]:
gb_train_agg =  gb_train.agg([np.min, np.max, np.std, np.mean, 
              mad, energy, iqr, entropy, skewness, kurtosis ])

In [73]:
gb_test_agg = gb_test.agg([np.min, np.max, np.std, np.mean, 
              mad, energy, iqr, entropy, skewness, kurtosis ])
gb_test_agg

acc_x                                                              \
          amin      amax       std      mean       mad    energy       iqr   
id                                                                           
3125 -1.564000 -0.275446  0.236232 -1.018731  0.144306  1.093525  0.300986   
3126 -1.929033  0.627571  0.539688 -0.522843  0.273615  0.564143  0.675293   
3127 -0.792916  2.972063  0.219934  0.506947  0.055507  0.305285  0.111048   
3128 -1.045889  0.337281  0.431713 -0.577603  0.095699  0.519690  0.812036   
3129 -2.153047  0.015642  0.305797 -0.738640  0.156308  0.638945  0.318067   
...        ...       ...       ...       ...       ...       ...       ...   
3902 -2.050254  0.427159  0.352604 -0.907299  0.106462  0.947315  0.222628   
3903 -1.709527  1.659451  0.663522 -0.608731  0.278003  0.810081  0.656250   
3904 -2.124959 -0.085249  0.252666 -0.753193  0.152712  0.631033  0.414637   
3905  0.536568  1.438345  0.164880  0.958903  0.136808  0.946635  0.275647   
3906 -1.534616  0.224194  0.323766 -0.781752  0.132731  0.715787  0.293332   

                                         acc_y                                \
        entropy  skewness   kurtosis      amin      amax       std      mean   
id                                                                             
3125   9.187356  0.479478  -0.108082 -0.470937  0.228040  0.091641 -0.019574   
3126   9.922442  0.781933  -0.439259 -0.200678  1.708743  0.333015  0.612161   
3127   9.234553  2.594860  42.977585  0.219008  1.941820  0.191485  0.903819   
3128   9.035201  0.631184  -1.294690 -1.294482 -0.258476  0.233601 -0.610557   
3129   9.094071 -0.333323   0.881997 -0.860883  1.562602  0.314294  0.182535   
...         ...       ...        ...       ...       ...       ...       ...   
3902   9.204437  1.743894   4.075400 -1.414874  3.057501  0.370471  0.488264   
3903  10.238501  1.472359   1.555368 -1.247963  1.549890  0.406506  0.371269   
3904   9.140955  0.078776   0.756878 -0.443533  1.236138  0.300552  0.199782   
3905   9.207353  0.055789  -0.739652 -0.580191  0.076427  0.120706 -0.320975   
3906   9.098697  1.067843   0.779222 -0.958351  0.144179  0.219289 -0.369244   

                                                                      acc_z  \
           mad    energy       iqr    entropy  skewness  kurtosis      amin   
id                                                                            
3125  0.052060  0.008767  0.105785  14.906724 -0.736383  1.782043 -0.573836   
3126  0.269067  0.485455  0.544843   9.020439  0.141358 -0.558259 -1.212052   
3127  0.142896  0.853493  0.286264   9.195669  0.048228  0.854547 -0.484614   
3128  0.112980  0.427258  0.289727   9.127681 -0.707660 -0.581285 -0.469924   
3129  0.191663  0.131935  0.415437  10.007257  0.243577  0.368275 -0.631258   
...        ...       ...       ...        ...       ...       ...       ...   
3902  0.116188  0.375421  0.243023   9.263642  0.545124  9.314337 -1.517656   
3903  0.247461  0.302812  0.500324   9.360408 -0.232367  1.006670 -0.996954   
3904  0.188725  0.130094  0.505337   9.338398  0.417754 -0.799295 -0.332901   
3905  0.077109  0.117571  0.152215   9.119033  0.350580 -0.025782 -0.774209   
3906  0.139629  0.184349  0.268191   9.013467 -0.036627 -0.336178 -0.539595   

                                                                             \
          amax       std      mean       mad    energy       iqr    entropy   
id                                                                            
3125  0.286182  0.174672 -0.232259  0.129298  0.084404  0.270002   9.094167   
3126  0.671876  0.383800 -0.071092  0.278699  0.152111  0.586406  18.288292   
3127  0.644154  0.152077 -0.141098  0.109579  0.042998  0.220560   9.396199   
3128  0.702574  0.326569  0.031486  0.046761  0.107461  0.459605  23.362589   
3129  1.037876  0.261848  0.401302  0.180134  0.229493  0.393349   9.083133   
...        ...       ...       ...       ...       ...  

In [36]:
train_ids = train.id.nunique()
test_ids = test.id.nunique()
train_df = [] 
for i in range(train_ids):
    train_df.append(train.iloc[600*i : 600*i + 600])
    
test_df = []
for i in range(test_ids):
    test_df.append(test.iloc[600*i : 600*i + 600 ])

In [68]:
# train x, y, z 각 features 간의 angle 
angle_acc_xy = []
for i in range(train_ids):
    xy = angleBet(train_df[i].acc_x, train_df[i].acc_y)
    angle_acc_xy = np.append(angle_acc_xy, xy)
    
angle_acc_xz = []
for i in range(train_ids):
    xz = angleBet(train_df[i].acc_x, train_df[i].acc_z)
    angle_acc_xz = np.append(angle_acc_xz, xz)

angle_acc_yz = []   
for i in range(train_ids):
    yz = angleBet(train_df[i].acc_y, train_df[i].acc_z)
    angle_acc_yz = np.append(angle_acc_yz, yz)

angle_gy_xy = []   
for i in range(train_ids):
    xy = angleBet(train_df[i].gy_x, train_df[i].gy_y)
    angle_gy_xy = np.append(angle_gy_xy, xy)

angle_gy_xz = []   
for i in range(train_ids):
    xz = angleBet(train_df[i].gy_x, train_df[i].gy_z)
    angle_gy_xz = np.append(angle_gy_xz, xz)      
    
angle_gy_yz = []   
for i in range(train_ids):
    yz = angleBet(train_df[i].gy_y, train_df[i].gy_z)
    angle_gy_yz = np.append(angle_gy_yz, yz)

In [74]:
# test x, y, z 각 features 간의 angle 
t_angle_acc_xy = []
t_angle_acc_xz = []
t_angle_acc_yz = [] 
t_angle_gy_xy = [] 
t_angle_gy_xz = [] 
t_angle_gy_yz = [] 

for i in range(test_ids):
    # acc x, y, z
    a_xy = angleBet(test_df[i].acc_x, test_df[i].acc_y)
    a_xz = angleBet(test_df[i].acc_x, test_df[i].acc_z)
    a_yz = angleBet(test_df[i].acc_y, test_df[i].acc_z)
    t_angle_acc_xy = np.append(t_angle_acc_xy, a_xy)
    t_angle_acc_xz = np.append(t_angle_acc_xz, a_xz)
    t_angle_acc_yz = np.append(t_angle_acc_yz, a_yz)
    
    # gy x, y, z
    g_xy = angleBet(test_df[i].gy_x, test_df[i].gy_y)
    g_xz = angleBet(test_df[i].gy_x, test_df[i].gy_z)
    g_yz = angleBet(test_df[i].gy_y, test_df[i].gy_z)
    t_angle_gy_xy = np.append(t_angle_gy_xy, g_xy)
    t_angle_gy_xz = np.append(t_angle_gy_xz, g_xz) 
    t_angle_gy_yz = np.append(t_angle_gy_yz, g_yz)

In [51]:
gb_train_agg.columns = ['_'.join(col) for col in gb_train_agg.columns]

In [75]:
gb_test_agg.columns = ['_'.join(col) for col in gb_test_agg.columns]

In [63]:
angles = {'angle_acc_xy' : angle_acc_xy, 
         'angle_acc_yz' : angle_acc_yz,
         'angle_acc_xz' : angle_acc_xz,
         'angle_gy_xy' : angle_gy_xy,
         'angle_gy_yz': angle_gy_yz,
         'angle_gy_xz': angle_gy_xz}

angles = pd.DataFrame(angles)

In [76]:
t_angles = {'angle_acc_xy' : t_angle_acc_xy, 
         'angle_acc_yz' : t_angle_acc_yz,
         'angle_acc_xz' : t_angle_acc_xz,
         'angle_gy_xy' : t_angle_gy_xy,
         'angle_gy_yz': t_angle_gy_yz,
         'angle_gy_xz': t_angle_gy_xz}

In [79]:
t_angles = pd.DataFrame(t_angles)

In [81]:
t_angles

,angle_acc_xy,angle_acc_yz,angle_acc_xz,angle_gy_xy,angle_gy_yz,angle_gy_xz
0,1.322458,1.689582,0.701889,1.341186,0.906052,1.530882
1,2.108076,1.835634,1.047148,2.061915,1.470409,1.873111
2,0.419246,2.415023,2.111600,1.831714,1.126729,2.159860
3,0.955650,1.667640,1.399086,0.989498,0.950927,1.562250
4,1.941617,1.005741,2.265470,1.473316,1.830280,1.822474
...,...,...,...,...,...,...
777,2.304674,2.388195,0.852669,2.119784,1.021408,1.651963
778,1.978621,2.093672,1.178977,2.013153,1.077192,1.824073
779,1.897408,0.657839,2.186990,1.443274,2.209882,1.643250
780,2.825397,0.489220,2.787207,1.932535,0.680150,1.488236


In [65]:
gb_train_df = pd.merge(gb_train_agg, angles, left_index = True,right_index=True)

In [98]:
gb_test_df = pd.merge(gb_test_agg.reset_index(), t_angles, left_index = True, right_index = True)
gb_test_df.set_index('id', inplace = True)

In [99]:
gb_test_df

,acc_x_amin,acc_x_amax,acc_x_std,acc_x_mean,acc_x_mad,acc_x_energy,acc_x_iqr,acc_x_entropy,acc_x_skewness,acc_x_kurtosis,acc_y_amin,acc_y_amax,acc_y_std,acc_y_mean,acc_y_mad,acc_y_energy,acc_y_iqr,acc_y_entropy,acc_y_skewness,acc_y_kurtosis,acc_z_amin,acc_z_amax,acc_z_std,acc_z_mean,acc_z_mad,acc_z_energy,acc_z_iqr,acc_z_entropy,acc_z_skewness,acc_z_kurtosis,gy_x_amin,gy_x_amax,gy_x_std,gy_x_mean,gy_x_mad,gy_x_energy,gy_x_iqr,gy_x_entropy,gy_x_skewness,gy_x_kurtosis,gy_y_amin,gy_y_amax,gy_y_std,gy_y_mean,gy_y_mad,gy_y_energy,gy_y_iqr,gy_y_entropy,gy_y_skewness,gy_y_kurtosis,gy_z_amin,gy_z_amax,gy_z_std,gy_z_mean,gy_z_mad,gy_z_energy,gy_z_iqr,gy_z_entropy,gy_z_skewness,gy_z_kurtosis,angle_acc_xy,angle_acc_yz,angle_acc_xz,angle_gy_xy,angle_gy_yz,angle_gy_xz
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3125,-1.564000,-0.275446,0.236232,-1.018731,0.144306,1.093525,0.300986,9.187356,0.479478,-0.108082,-0.470937,0.228040,0.091641,-0.019574,0.052060,0.008767,0.105785,14.906724,-0.736383,1.782043,-0.573836,0.286182,0.174672,-0.232259,0.129298,0.084404,0.270002,9.094167,0.439717,-0.671472,-50.429364,56.953059,12.897967,-3.185128,8.459255,176.225332,16.851179,15.269663,0.372202,0.955856,-81.607713,96.185341,31.993022,2.731872,19.330232,1029.310658,38.646525,26.943492,0.284653,0.269311,-35.446915,49.981455,12.251648,-2.000683,7.265558,153.855430,14.588516,18.208013,0.284257,1.305175,1.322458,1.689582,0.701889,1.341186,0.906052,1.530882
3126,-1.929033,0.627571,0.539688,-0.522843,0.273615,0.564143,0.675293,9.922442,0.781933,-0.439259,-0.200678,1.708743,0.333015,0.612161,0.269067,0.485455,0.544843,9.020439,0.141358,-0.558259,-1.212052,0.671876,0.383800,-0.071092,0.278699,0.152111,0.586406,18.288292,0.201677,-0.790734,-273.572486,132.830402,51.625096,-17.741526,29.917241,2975.470379,60.361553,11.839251,-1.282840,5.115425,-97.100707,241.240196,45.706311,6.974772,20.171896,2134.232488,40.405884,16.832198,1.237802,2.981170,-147.597574,169.417650,61.604867,-3.604579,34.751055,3801.827383,73.081878,37.469996,0.461218,0.143382,2.108076,1.835634,1.047148,2.061915,1.470409,1.873111
3127,-0.792916,2.972063,0.219934,0.506947,0.055507,0.305285,0.111048,9.234553,2.594860,42.977585,0.219008,1.941820,0.191485,0.903819,0.142896,0.853493,0.286264,9.195669,0.048228,0.854547,-0.484614,0.644154,0.152077,-0.141098,0.109579,0.042998,0.220560,9.396199,0.344382,0.438891,-99.799971,160.426058,22.770845,-2.179745,12.046792,522.398474,26.062826,24.022945,0.864992,9.003908,-98.420987,74.530763,13.467885,-2.251452,6.304654,186.150669,12.360286,16.349451,-0.948438,13.756362,-154.477074,97.211730,23.041463,-0.393175,12.587140,530.178750,26.208357,71.475445,-1.100760,9.102643,0.419246,2.415023,2.111600,1.831714,1.126729,2.159860
3128,-1.045889,0.337281,0.431713,-0.577603,0.095699,0.519690,0.812036,9.035201,0.631184,-1.294690,-1.294482,-0.258476,0.233601,-0.610557,0.112980,0.427258,0.289727,9.127681,-0.707660,-0.581285,-0.469924,0.702574,0.326569,0.031486,0.046761,0.107461,0.459605,23.362589,0.766024,-0.573780,-229.072919,119.527887,42.818157,0.808579,7.079276,1830.992674,14.533337,49.977913,-1.908157,10.017156,-168.031080,118.268797,45.069932,-2.984969,9.506666,2036.823334,19.354142,22.943135,-0.735070,3.751920,-117.297766,167.860762,37.967372,-0.024318,8.798729,1439.119422,17.773898,-642.954107,0.830138,5.279371,0.955650,1.667640,1.399086,0.989498,0.950927,1.562250
3129,-2.153047,0.015642,0.305797,-0.738640,0.156308,0.638945,0.318067,9.094071,-0.333323,0.881997,-0.860883,1.562602,0.314294,0.182535,0.191663,0.131935,0.415437,10.007257,0.243577,0.368275,-0.631258,1.037876,0.261848,0.401302,0.180134,0.229493,0.393349,9.083133,-0.511312,0.311924,-345.447240,366.167357,92.301963,-1.431538,34.337657,8507.502231,67.570154,68.764861,0.231276,3.117313,-223.475411,226.728939,67.911174,1.442366,33.231523,4606.321460,66.056762,59.967690,-0.005470,1.178294,-125.598600,138.130133,43.353007,5.745498,21.721165,1909.361472,41.611788,19.659931,0.042255,0.937784,1.9

In [100]:
gb_train_df

,acc_x_amin,acc_x_amax,acc_x_std,acc_x_mean,acc_x_mad,acc_x_energy,acc_x_iqr,acc_x_entropy,acc_x_skewness,acc_x_kurtosis,acc_y_amin,acc_y_amax,acc_y_std,acc_y_mean,acc_y_mad,acc_y_energy,acc_y_iqr,acc_y_entropy,acc_y_skewness,acc_y_kurtosis,acc_z_amin,acc_z_amax,acc_z_std,acc_z_mean,acc_z_mad,acc_z_energy,acc_z_iqr,acc_z_entropy,acc_z_skewness,acc_z_kurtosis,gy_x_amin,gy_x_amax,gy_x_std,gy_x_mean,gy_x_mad,gy_x_energy,gy_x_iqr,gy_x_entropy,gy_x_skewness,gy_x_kurtosis,gy_y_amin,gy_y_amax,gy_y_std,gy_y_mean,gy_y_mad,gy_y_energy,gy_y_iqr,gy_y_entropy,gy_y_skewness,gy_y_kurtosis,gy_z_amin,gy_z_amax,gy_z_std,gy_z_mean,gy_z_mad,gy_z_energy,gy_z_iqr,gy_z_entropy,gy_z_skewness,gy_z_kurtosis,angle_acc_xy,angle_acc_yz,angle_acc_xz,angle_gy_xy,angle_gy_yz,angle_gy_xz
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.591940,1.344268,0.191479,0.931329,0.164385,0.903977,0.332775,9.198210,0.106076,-1.161667,-0.624113,0.176871,0.177131,-0.218471,0.138511,0.079053,0.291416,8.996050,0.054884,-0.881409,-0.786336,-0.054876,0.135131,-0.370422,0.082231,0.155442,0.178503,9.134392,-0.595532,-0.148805,-46.254836,31.644123,13.284216,-1.865269,8.773819,179.655497,17.543005,20.473157,-0.398648,0.211136,-85.887677,69.847244,24.300479,-3.359506,13.181243,600.815385,26.104757,19.698210,-0.204916,0.732569,-79.930029,55.953827,25.275185,1.182107,14.828431,639.167632,29.409880,41.633168,-0.577464,0.366905,2.609469,0.953185,2.648885,1.381793,0.643786,1.652276
1,-2.156208,1.234020,0.495528,-0.766580,0.321762,0.832784,0.674126,8.979081,-0.202959,0.229016,-1.295598,0.700065,0.336415,-0.317258,0.232913,0.213639,0.541698,9.050932,-0.145735,-0.589686,-1.019531,0.888661,0.499395,-0.004223,0.382460,0.248998,0.735812,108.008567,-0.137812,-0.764662,-325.328531,286.624363,79.244561,11.071600,37.677737,6391.814635,77.347142,18.846755,-0.345188,2.101102,-315.096003,389.608060,96.005289,1.740475,27.991032,9204.683071,57.265455,64.732472,-0.486011,2.828346,-270.980823,340.170199,75.545343,1.393294,34.252342,5699.528320,68.036458,61.691045,0.013679,3.906192,1.142600,1.853288,1.335142,1.066784,0.990918,1.395510
2,-1.142847,1.219836,0.711972,0.039836,0.644637,0.507646,1.425141,43.929103,-0.180743,-1.531553,-0.690990,0.650645,0.147127,-0.082403,0.061567,0.028401,0.127779,9.783002,-0.409084,3.478554,0.073846,1.332992,0.248807,0.626012,0.214759,0.453692,0.427353,9.109934,0.091623,-1.033791,-164.779067,73.525082,25.422926,-8.472951,11.236095,717.038840,22.371612,12.195904,-0.705518,4.062259,-249.953944,297.320834,118.956646,0.597877,97.114425,14127.456660,207.728132,140.379325,-0.067697,-0.958142,-44.192071,55.642836,13.920337,3.053291,7.657415,202.775403,15.633852,15.738669,0.030235,1.170665,1.764720,2.089987,1.620481,1.614682,1.190557,1.605095
3,-1.417751,-0.622250,0.130899,-0.887702,0.098201,0.805121,0.197374,9.213446,-0.606239,0.662947,-0.540827,0.283721,0.194008,-0.087668,0.138254,0.045262,0.288082,11.040507,-0.388269,-0.739255,-0.257124,0.598814,0.205882,0.227357,0.186759,0.094008,0.392053,9.118189,-0.141053,-1.007299,-69.419166,192.765368,42.928860,17.744167,24.029710,2154.671009,49.473159,10.589838,0.985137,1.076992,-82.537304,159.083788,36.953466,4.800931,18.497637,1386.331628,36.814877,19.149927,0.936520,2.187724,-85.600536,56.456908,23.647153,-5.869898,12.912456,592.711564,26.940305,14.138010,-0.323481,0.420130,1.143310,1.618612,2.322312,1.223719,1.347102,2.134248
4,-2.429109,0.599720,0.495170,-0.659018,0.150869,0.679090,0.613075,9.183900,0.491559,-0.175348,-2.055076,1.724782,0.570305,-0.337067,0.332205,0.438319,0.990325,9.787168,0.087540,-0.083092,-1.250483,2.678034,0.389646,0.202758,0.080679,0.192682,0.194180,11.069722,0.218454,7.987059,-769.076518,613.972600,108.258866,-4.819638,8.283215,11723.677703,16.786365,21.945725,-1.391414,18.012546,-243.909948,284.952954,60.514531,9.651713,6.905686,3749.060676,14.398692,11.758296,1.531164,8.916135,-270.581913,221.015193,46.148326,4.453382,6.873766,2145.951184,16.057779,17.098861,-0.157801,8.118891,1.549291

In [101]:
y = train_label['label']

In [102]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import time
import datetime

In [105]:
X_train, X_test, y_train, y_test = train_test_split(gb_train_df, y, 
                                                   test_size = 0.2, random_state = 42)
evals = [(X_test.values, y_test)]

xgb = XGBClassifier(n_estimators = 400, learning_rate = 0.05,
                   max_depth = 8, objective = 'multi:softprob', random_state = 0)

In [106]:
start = time.time()
xgb.fit(gb_train_df.values, y, eval_metric = 'mlogloss', eval_set = evals,
       early_stopping_rounds = 100)

sec = time.time() -start 
print('학습시간 : ', str(datetime.timedelta(seconds = sec)).split('.'))

[0]	validation_0-mlogloss:3.30499
[1]	validation_0-mlogloss:2.97993
[2]	validation_0-mlogloss:2.73191
[3]	validation_0-mlogloss:2.53525
[4]	validation_0-mlogloss:2.36296
[5]	validation_0-mlogloss:2.21667
[6]	validation_0-mlogloss:2.08913
[7]	validation_0-mlogloss:1.97301
[8]	validation_0-mlogloss:1.86661
[9]	validation_0-mlogloss:1.76992
[10]	validation_0-mlogloss:1.68211
[11]	validation_0-mlogloss:1.60188
[12]	validation_0-mlogloss:1.52606
[13]	validation_0-mlogloss:1.45616
[14]	validation_0-mlogloss:1.39173
[15]	validation_0-mlogloss:1.33105
[16]	validation_0-mlogloss:1.27378
[17]	validation_0-mlogloss:1.21886
[18]	validation_0-mlogloss:1.16791
[19]	validation_0-mlogloss:1.11997
[20]	validation_0-mlogloss:1.07438
[21]	validation_0-mlogloss:1.03089
[22]	validation_0-mlogloss:0.98983
[23]	validation_0-mlogloss:0.95004
[24]	validation_0-mlogloss:0.91230
[25]	validation_0-mlogloss:0.87648
[26]	validation_0-mlogloss:0.84283
[27]	validation_0-mlogloss:0.80984
[28]	validation_0-mlogloss:0.7

In [107]:
y_pred = xgb.predict_proba(gb_test_df.values)
y_pred.shape

(782, 61)

In [108]:
submission.iloc[:,1:] = y_pred
submission.to_csv('submission파일/xgb_ha_1.csv', index = False)

제출 결과 값 : 0.8791158902

UCI 에서 제공하던 행동 데이터의 레이블은 분포는 왜곡되지 않고, 비교적 고르게 분포되어 있었다. 그러나 해당 문제의 경우 레이블의 분포가 Non_Exercise에 유난히 집중되어 있다. 그로 인해 데이터가 적은 레이블에 대한 학습이 제대로 이루어지지 않을 가능성이 존재한다. 

In [110]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 0)

X_over, y_over = smote.fit_sample(gb_train_df, y)

In [111]:
X_over.shape

(92598, 66)

In [112]:
X_train, X_test, y_train, y_test = train_test_split(X_over, y_over, test_size = 0.2,
                                                   random_state = 42)

evals = [(X_test.values, y_test)]

In [113]:
start = time.time()
xgb.fit(X_over.values, y_over, eval_metric = 'mlogloss', eval_set = evals,
       early_stopping_rounds = 100)

sec = time.time() -start 
print('학습시간 : ', str(datetime.timedelta(seconds = sec)).split('.'))

[0]	validation_0-mlogloss:2.88677
[1]	validation_0-mlogloss:2.50026
[2]	validation_0-mlogloss:2.23356
[3]	validation_0-mlogloss:2.02827
[4]	validation_0-mlogloss:1.86110
[5]	validation_0-mlogloss:1.72011
[6]	validation_0-mlogloss:1.59834
[7]	validation_0-mlogloss:1.49066
[8]	validation_0-mlogloss:1.39525
[9]	validation_0-mlogloss:1.30911
[10]	validation_0-mlogloss:1.23126
[11]	validation_0-mlogloss:1.15985
[12]	validation_0-mlogloss:1.09428
[13]	validation_0-mlogloss:1.03378
[14]	validation_0-mlogloss:0.97794
[15]	validation_0-mlogloss:0.92612
[16]	validation_0-mlogloss:0.87767
[17]	validation_0-mlogloss:0.83249
[18]	validation_0-mlogloss:0.79035
[19]	validation_0-mlogloss:0.75062
[20]	validation_0-mlogloss:0.71335
[21]	validation_0-mlogloss:0.67826
[22]	validation_0-mlogloss:0.64529
[23]	validation_0-mlogloss:0.61408
[24]	validation_0-mlogloss:0.58462
[25]	validation_0-mlogloss:0.55681
[26]	validation_0-mlogloss:0.53041
[27]	validation_0-mlogloss:0.50564
[28]	validation_0-mlogloss:0.4

In [114]:
y_pred = xgb.predict_proba(gb_test_df.values)
submission.iloc[:,1:] =y_pred
submission.to_csv('submission파일/over_xgb_ha.csv', index = False)

제출결과값 : 0.9255561507

## features Scaling 

In [116]:
from sklearn.preprocessing import StandardScaler

In [117]:
scaler = StandardScaler()

In [122]:
scaled_train = scaler.fit_transform(gb_train_df)

In [124]:
scaled_train.shape

(3125, 66)

In [126]:
X_train, X_test, y_train, y_test = train_test_split(scaled_train, y, test_size = 0.2,
                                                   random_state = 42)
evals = [(X_test, y_test)]

In [127]:
start = time.time()
xgb.fit(scaled_train, y, eval_metric = 'mlogloss', eval_set = evals,
       early_stopping_rounds = 100)

sec = time.time() -start 
print('학습시간 : ', str(datetime.timedelta(seconds = sec)).split('.'))

[0]	validation_0-mlogloss:3.30499
[1]	validation_0-mlogloss:2.97993
[2]	validation_0-mlogloss:2.73191
[3]	validation_0-mlogloss:2.53525
[4]	validation_0-mlogloss:2.36296
[5]	validation_0-mlogloss:2.21667
[6]	validation_0-mlogloss:2.08913
[7]	validation_0-mlogloss:1.97301
[8]	validation_0-mlogloss:1.86661
[9]	validation_0-mlogloss:1.76991
[10]	validation_0-mlogloss:1.68219
[11]	validation_0-mlogloss:1.60187
[12]	validation_0-mlogloss:1.52605
[13]	validation_0-mlogloss:1.45612
[14]	validation_0-mlogloss:1.39169
[15]	validation_0-mlogloss:1.33106
[16]	validation_0-mlogloss:1.27382
[17]	validation_0-mlogloss:1.21889
[18]	validation_0-mlogloss:1.16806
[19]	validation_0-mlogloss:1.11990
[20]	validation_0-mlogloss:1.07436
[21]	validation_0-mlogloss:1.03074
[22]	validation_0-mlogloss:0.98958
[23]	validation_0-mlogloss:0.94979
[24]	validation_0-mlogloss:0.91226
[25]	validation_0-mlogloss:0.87701
[26]	validation_0-mlogloss:0.84248
[27]	validation_0-mlogloss:0.81005
[28]	validation_0-mlogloss:0.7

In [131]:
scaled_test = scaler.fit_transform(gb_test_df)
y_pred = xgb.predict_proba(scaled_test)
y_pred.shape

(782, 61)

In [132]:
submission.iloc[:,1:] = y_pred
submission.to_csv('submission파일/xgb_scaled.csv', index = False)

제출결과값 : 1.1129363806

In [134]:
scaler = StandardScaler()
scaled_over_df = scaler.fit_transform(X_over)

In [135]:
scaled_over_df.shape

(92598, 66)

In [136]:
X_train, X_test, y_train, y_test = train_test_split(scaled_over_df, y_over, test_size = 0.2,
                                                   random_state = 42)
evals = [(X_test, y_test)]

In [138]:
start = time.time()
xgb.fit(scaled_over_df, y_over, eval_metric = 'mlogloss', eval_set = evals,
       early_stopping_rounds = 100)

sec = time.time() -start 
print('학습시간 : ', str(datetime.timedelta(seconds = sec)).split('.'))

[0]	validation_0-mlogloss:2.88677
[1]	validation_0-mlogloss:2.50026
[2]	validation_0-mlogloss:2.23356
[3]	validation_0-mlogloss:2.02827
[4]	validation_0-mlogloss:1.86110
[5]	validation_0-mlogloss:1.72011
[6]	validation_0-mlogloss:1.59834
[7]	validation_0-mlogloss:1.49066
[8]	validation_0-mlogloss:1.39525
[9]	validation_0-mlogloss:1.30911
[10]	validation_0-mlogloss:1.23126
[11]	validation_0-mlogloss:1.15985
[12]	validation_0-mlogloss:1.09428
[13]	validation_0-mlogloss:1.03378
[14]	validation_0-mlogloss:0.97794
[15]	validation_0-mlogloss:0.92612
[16]	validation_0-mlogloss:0.87768
[17]	validation_0-mlogloss:0.83249
[18]	validation_0-mlogloss:0.79036
[19]	validation_0-mlogloss:0.75062
[20]	validation_0-mlogloss:0.71335
[21]	validation_0-mlogloss:0.67826
[22]	validation_0-mlogloss:0.64529
[23]	validation_0-mlogloss:0.61408
[24]	validation_0-mlogloss:0.58462
[25]	validation_0-mlogloss:0.55682
[26]	validation_0-mlogloss:0.53042
[27]	validation_0-mlogloss:0.50563
[28]	validation_0-mlogloss:0.4

In [139]:
scaled_test = scaler.fit_transform(gb_test_df)
y_pred = xgb.predict_proba(scaled_test)
submission.iloc[:,1:] = y_pred
submission.to_csv('submission파일/over_xgb_scaled.csv', index = False)

제출 결과값 : 1.3195157804